### script to extract ortholog variants

###### INIT

In [136]:
import numpy as np
import sys
import copy
import os
from os.path import isfile, join, isdir

og_file = "./Output/OrthologousGroups.txt"
mapping_file = "./Output/Map-SeqNum-ID.txt"
PR_folder = "./Output/PairwiseOrthologs/"

###### Do mapping oma

In [137]:
class Map_species_id(object):
	'''
	Object to map species and genes from the Map-SeqNum-ID.txt in the OMA standalone Output folder.
	--| SPECIES  int_id  ext_id |--
	'''

	def create_map(self, _inputfile):
		map_dict = {}

		data = np.genfromtxt(_inputfile, dtype=None , delimiter="\t", usecols=(0,1,2))
		for line in data:
			map_dict.setdefault(line[0],[]).append([line[1],line[2]])
		return map_dict

	def __init__(self, _inputfile):
		self.map = self.create_map(_inputfile)

	def get_extid(self,query_id, query_species):
		'''
		return external id using species + internal id
		:param query_id:
		:param query_species:
		:return:
		'''
		for species,entries in self.map.iteritems():
			if query_species == species:
				for entry in entries:
					if entry[0] == int(query_id):
						return entry[1]

	def get_omaid(self,query_id, query_species):
		'''
		return internal id using species + external id
		:param query_omaid:
		:param query_species:
		:return:
		'''
		for species,entries in self.map.iteritems():
			if query_species == species:
				for entry in entries:
					if entry[1] == query_id:
						return entry[0]

	def get_species_with_ext(self,query_extid):
		'''
		get the species related to an external id
		:param query_extid:
		:return:
		'''
		for species,entries in self.map.iteritems():
				for entry in entries:
					if query_extid == entry[1]:
						return species

	def get_species_with_int(self,query_intid):
		'''
		get the species related to an internal id
		:param query_intid:
		:return:
		'''
		for species,entries in self.map.iteritems():
				for entry in entries:
					if query_intid == entry[0]:
						return species


In [138]:
map_id = Map_species_id(mapping_file)

###### Read OG file

In [139]:

# og_dict[keys|OG_id] -> values|dict[keys|species] -> values|list(gene_id)
og_dict = {} 
omaid_2_ogid = {}
 
with open(og_file) as f:
    
        content = f.readlines()
        cpt = 0
        
        # for each og
        for line in content:
            cpt +=1 
            if cpt % 100 == 0:
                print(str(cpt) + " OGs computed/ " + str(len(content)-4))
            # if line not comment
            if line[0] != "#":
                
                #split by '\t' to have each element in an list
                line = line.rstrip('\n')
                spl =  line.split('\t')
                
                # get og id and add it to og_dict
                og_id = str(spl[0])
                og_dict[og_id]={}
                
                # for each gene in the og
                for gene in spl[1:]:
                    
                    # get its species
                    sp = str(map_id.get_species_with_ext(gene[4:]))
                    
                    if sp != gene[:3]:
                        sys.exit("Error of species mapping in the OG " + og_id + " for gene " + gene)
                    
                    # get its oma id (since internal we are working with them)
                    oma_id = str(map_id.get_omaid(str(gene[4:]), sp))
                    
                    # add the gene with the corresponding species key
                    og_dict[og_id].setdefault(sp,[]).append(oma_id)
                    
                    omaid_2_ogid[str(sp) + str(oma_id)] = str(og_id)



100 OGs computed/ 13396
200 OGs computed/ 13396
300 OGs computed/ 13396
400 OGs computed/ 13396
500 OGs computed/ 13396
600 OGs computed/ 13396
700 OGs computed/ 13396
800 OGs computed/ 13396
900 OGs computed/ 13396
1000 OGs computed/ 13396
1100 OGs computed/ 13396
1200 OGs computed/ 13396
1300 OGs computed/ 13396
1400 OGs computed/ 13396
1500 OGs computed/ 13396
1600 OGs computed/ 13396
1700 OGs computed/ 13396
1800 OGs computed/ 13396
1900 OGs computed/ 13396
2000 OGs computed/ 13396
2100 OGs computed/ 13396
2200 OGs computed/ 13396
2300 OGs computed/ 13396
2400 OGs computed/ 13396
2500 OGs computed/ 13396
2600 OGs computed/ 13396
2700 OGs computed/ 13396
2800 OGs computed/ 13396
2900 OGs computed/ 13396
3000 OGs computed/ 13396
3100 OGs computed/ 13396
3200 OGs computed/ 13396
3300 OGs computed/ 13396
3400 OGs computed/ 13396
3500 OGs computed/ 13396
3600 OGs computed/ 13396
3700 OGs computed/ 13396
3800 OGs computed/ 13396
3900 OGs computed/ 13396
4000 OGs computed/ 13396
4100 OGs 

In [193]:
og_dict_plus = copy.deepcopy(og_dict)

###### Get list species + PR files

In [141]:
def get_list_files(mypath):
    onlyfiles = [ f for f in os.listdir(mypath) if isfile(join(mypath,f)) ]
    return onlyfiles

def get_list_species_from_standalone_folder(input_folder):
    '''
    return species name from the file names (species1-species2.ext) of the pairwise folder
    :param input_folder:
    :return:
    '''
    list_species = []
    for file in get_list_files(input_folder):
        file_name_no_ext = file.split(os.extsep, 1)[0]
        array_name = file_name_no_ext.split("-")
        for species_name in array_name:
            list_species.append(species_name)
    return list(set(list_species))

# Extract list of species from the PR folder
list_species = get_list_species_from_standalone_folder(PR_folder)

# Get the list of files
PR_files = get_list_files(PR_folder)


###### Add many:many and 1:many link into OG plus

In [194]:
import collections

"""UnionFind.py

Union-find data structure. Based on Josiah Carlson's code,
http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/215912
with significant additional changes by D. Eppstein and
Adrian Altenhoff.
"""

class UnionFind(object):
    """Union-find data structure.

    Each unionFind instance X maintains a family of disjoint sets of
    hashable objects, supporting the following two methods:

    - X[item] returns a name for the set containing the given item.
      Each set is named by an arbitrarily-chosen one of its members; as
      long as the set remains unchanged it will keep the same name. If
      the item is not yet part of a set in X, a new singleton set is
      created for it.

    - X.union(item1, item2, ...) merges the sets containing each item
      into a single larger set.  If any item is not yet part of a set
      in X, it is added to X as one of the members of the merged set.
    """

    def __init__(self, elements=None):
        """Create a new empty union-find structure."""
        self.weights = {}
        self.parents = {}
        if not elements is None:
            for elem in iter(elements):
                self.parents[elem] = elem
                self.weights[elem] = 1

    def __getitem__(self, obj):
        return self.find(obj)

    def find(self,obj):
        """Find and return the name of the set containing the obj."""

        # check for previously unknown obj. If unknown, add it 
        # as a new cluster
        if obj not in self.parents:
            self.parents[obj] = obj
            self.weights[obj] = 1
            return obj

        # find path of objects leading to the root
        path = [obj]
        root = self.parents[obj]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        """Iterate through all items ever found or unioned by this structure."""
        return iter(self.parents)

    def union(self, *objects):
        """Find the sets containing the objects and merge them all."""
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r],r) for r in roots], key=lambda x:x[0])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest

    def get_components(self):
        """return a list of sets corresponding to the connected
        components of the structure."""
        comp_dict = collections.defaultdict(set)
        for elem in iter(self):
            comp_dict[self[elem]].add(elem)
        comp = list(comp_dict.values())
        return comp
    
modified_og = []

cpt = 0 
for pr_file in PR_files:
    
    cpt += 1
    print str(cpt) + "/" + str(len(PR_files))
    
    # get the name of the genomes pairs
    file_name_no_ext = pr_file.split(os.extsep, 1)[0]
    array_name = file_name_no_ext.split("-")
    
    # get all pr relations
    prs =  np.genfromtxt(PR_folder + pr_file, dtype=None, comments="#", delimiter="\t", usecols=(0,1))
    
    # get all CC in the PR graph
    cc_prs = UnionFind()
    for pr in prs:
        gene_l = array_name[0]+str(pr[0])
        gene_r = array_name[1]+str(pr[1])
        cc_prs.union(gene_l, gene_r)
     
    # Look in each CC if at least one of each genome is present in the same OG
    for cc in cc_prs.get_components():
        
        if len(cc) > 2:
            OG_sp = {}            
            for gid in cc:
                if gid in omaid_2_ogid.keys():
                    OG_sp.setdefault(omaid_2_ogid[gid],[]).append(gid[:3])
            for ogid, listsp in OG_sp.items():
                if len(list(set(listsp))) == 2 :
                    
                    modified_og.append(ogid)
                    for g2add in cc:
                        if g2add[3:] not in og_dict_plus[ogid][g2add[:3]]:
                            og_dict_plus[ogid][g2add[:3]].append(g2add[3:])
    
    modified_og = list(set(modified_og))

1/45
2/45
3/45
4/45
5/45
6/45
7/45
8/45
9/45
10/45
11/45
12/45
13/45
14/45
15/45
16/45
17/45
18/45
19/45
20/45
21/45
22/45
23/45
24/45
25/45
26/45
27/45
28/45
29/45
30/45
31/45
32/45
33/45
34/45
35/45
36/45
37/45
38/45
39/45
40/45
41/45
42/45
43/45
44/45
45/45


###### Write OG plus into file

In [197]:
f = open('./OrthologousGroupsPlus.txt', 'w+')
fl = open('./OrthologousGroupsPlusLongId.txt', 'w+')
for ogi in og_dict_plus.keys():
    f.write(ogi)
    fl.write(ogi)
    for spec in sorted(og_dict_plus[ogi].keys()):
        for gid in og_dict_plus[ogi][spec]:
            f.write("\t" + spec+gid )

            fl.write("\t" + map_id.get_extid(gid,spec) )
    f.write("\n")
    fl.write("\n")
f.close()
fl.close()

###### Write occupancy GO by species

In [202]:
fs = open('./OrthologousGroupsPlusStat.txt', 'w+')

fs.write("#OGid")
for s in sorted(list_species):
    fs.write("\t" + s) 
fs.write("\n")

for ogi in og_dict_plus.keys():
    fs.write(ogi)
    
    for s in sorted(list_species):
        if s in og_dict_plus[ogi].keys():
            fs.write( "\t" + str(len(og_dict_plus[ogi][s])))
        else:
            fs.write("\t" + str(0) )
    fs.write("\n")

fs.close()


SyntaxError: invalid syntax (<ipython-input-202-4c1202d27378>, line 15)

###### Write only modified OGplus and stat

In [199]:
f = open('./OrthologousGroupsPlusOnlyModified.txt', 'w+')
fl = open('./OrthologousGroupsPlusLongIdOnlyModified.txt', 'w+')
for ogi in og_dict_plus.keys():
    if ogi in modified_og:
        f.write(ogi)
        fl.write(ogi)
        for spec in sorted(og_dict_plus[ogi].keys()):
            for gid in og_dict_plus[ogi][spec]:
                f.write("\t" + spec+gid )
                fl.write("\t" + map_id.get_extid(gid,spec) )
        f.write("\n")
        fl.write("\n")
f.close()
fl.close()

###### Write occupancy of species in each OG only for modified OG

In [201]:
fs = open('./OrthologousGroupsPlusStatOnlyModified.txt', 'w+')

fs.write("#OGid")
for s in sorted(list_species):
    fs.write("\t" + s) 
fs.write("\n")

for ogi in og_dict_plus.keys():
    if ogi in modified_og:
        fs.write(ogi)

        for s in sorted(list_species):
            if s in og_dict_plus[ogi].keys():
                fs.write( "\t" + str(len(og_dict_plus[ogi][s])) )
            else:
                fs.write( "\t" + str(0))
        fs.write("\n")

fs.close()

In [192]:
x = "tbe12345"
print x[3:]
print x[4:]


12345
2345
